In [1]:
import numpy as np
import scipy as sc

from numpy import cos
from numpy import arccos
from numpy import sin
from numpy import pi
from numpy import inf

from scipy.optimize import Bounds

In [2]:
path_to_ligand = r"C:\study\bioinf-bsu\binding_optimization\14010\02_cleared_9.pdbqt"
path_to_protein = r"C:\study\bioinf-bsu\binding_optimization\gp120.pdbqt"

In [3]:
def getCoordsFromPDB(input_path, mode):
    with open(input_path, 'r') as fin:
        out_name = list(input_path)[:-6]
        out_name = ''.join(out_name)
        fout = open(out_name + "_crds.txt", "w")
        
        data = fin.readlines()
        atoms = []
        for s in data:
            if (list(s)[0:4] == ['A', 'T', 'O', 'M']):
                words = s.split(" ")
                while '' in words:
                    words.remove('')
                if(mode == "prot"):
                    coords = words[6:9]
                else:
                    coords = words[5:8]
                coords_array = []
                for c in coords:
                    fout.write(c + " ")
                    coords_array.append(float(c))
                fout.write('\n')
                atoms.append(coords_array)
        
        fout.close()
        return atoms

In [4]:
def generateNewPDB(input_path, new_coords):
    with open(input_path, 'r') as fin:
        out_name = list(input_path)[:-6]
        out_name = ''.join(out_name)
        fout = open(out_name + "_optimized.pdbqt", "w")
        
        data = fin.readlines()
        atoms = []
        index = 0
        for s in data:
            if (list(s)[0:4] == ['A', 'T', 'O', 'M']):
                words = s.split(" ")
                while '' in words:
                    words.remove('')
                
                new_coords[index][0] = "{:.3f}".format(new_coords[index][0])
                new_coords[index][1] = "{:.3f}".format(new_coords[index][1])
                new_coords[index][2] = "{:.3f}".format(new_coords[index][2])
                
                coords = words[5:8]
                for i in range(3):
                    delta = len(coords[i]) - len(new_coords[index][i])
                    if (delta < 0):
                        for k in range(-delta):
                            coords[i] = ''.join([' '] + list(coords[i]))
                    if (delta > 0):
                        for k in range(delta):
                            new_coords[index][i] = ''.join([' '] + list(new_coords[index][i]))
                            
                s = s.replace(coords[0], new_coords[index][0])
                s = s.replace(coords[1], new_coords[index][1])
                s = s.replace(coords[2], new_coords[index][2])
                index += 1
            fout.write(s)
        
        fout.close()

In [5]:
ligand = getCoordsFromPDB(path_to_ligand, mode = "lig")
ligand_copy = getCoordsFromPDB(path_to_ligand, mode = "lig")
ligand_copy2 = getCoordsFromPDB(path_to_ligand, mode = "lig")

protein = getCoordsFromPDB(path_to_protein, mode = "prot")

In [6]:
def dist(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def normalize_vector(vector):
    length = dist(np.array([0, 0, 0]), vector)
    return vector / length

def angle_between_three_points(p1, p2, p3):
    vector1 = normalize_vector(np.array(p1) - np.array(p2))
    vector2 = normalize_vector(np.array(p3) - np.array(p2))
    
    return np.sum(vector1 * vector2)

In [7]:
def inBox(atom, box):
    if(box[0] > box[3]):
        tmp = box[0]
        box[0] = box[3]
        box[3] = tmp
    if(box[1] > box[4]):
        tmp = box[1]
        box[1] = box[4]
        box[4] = tmp
    if(box[2] > box[5]):
        tmp = box[2]
        box[2] = box[5]
        box[5] = tmp
        
    if(atom[0] > box[0] and 
       atom[0] < box[3] and 
       atom[1] > box[1] and 
       atom[1] < box[4] and 
       atom[2] > box[2] and 
       atom[2] < box[5]):
        return True
    else:
        return False

In [8]:
def filterProteinAtoms(protein, box):
    filtered_atoms = []
    for atom in protein:
        if(inBox(atom, box)):
            filtered_atoms.append(atom)
            
    print("Filtering finished. Atoms left: " + str(len(filtered_atoms)))
            
    return filtered_atoms

In [9]:
box = [21, -17, 73, 37, -3, 87]
protein = filterProteinAtoms(protein, box)

Filtering finished. Atoms left: 150


In [10]:
bonded_atoms_p = [[31.627, -6.185, 77.800],
                  [35.463, -5.785, 79.782],  
                  [35.573, -6.364, 78.950], 
                  [34.111, -5.480, 80.254],
                  [33.917, -5.533, 81.767], 
                  [34.601, -6.284, 82.470], 
                  [33.118, -6.431, 79.603], 
                  [32.804, -6.052, 78.182], 
                  [33.720, -5.637, 77.432]]

bonded_atoms_l = [[29.870, -14.262, 77.478],
                  [29.517, -14.009, 76.595]]

protein_binding_atom = [31.627, -6.185, 77.800]

In [11]:
#first
#ligand_binding_atom_basic = [29.870, -14.262, 77.478]
#h_atom_basic = [29.517, -14.009, 76.595]

#31-11021
#ligand_binding_atom_basic = [30.235, -8.234, 77.038] #h
#h_atom_basic = [30.291, -8.529, 77.975] #n

#07-11021
#ligand_binding_atom_basic = [31.382,  -8.228,  78.692] #h
#h_atom_basic = [31.117,  -8.846,  79.411] #n

#06-11021
#ligand_binding_atom_basic = [31.323,  -8.231,  78.740] #h
#h_atom_basic = [31.063,  -8.875,  79.438] #n

#17-11021
#ligand_binding_atom_basic = [30.471,  -8.379,  78.083] #h
#h_atom_basic = [30.398,  -8.796,  78.972] #n

#05-11021-1it
#ligand_binding_atom_basic = [31.146,  -9.227,  76.848] #h
#h_atom_basic = [31.011,  -9.302,  77.821] #n

#20-11021
#ligand_binding_atom_basic = [31.670,  -8.333,  79.159] #h
#h_atom_basic = [30.833,  -8.788,  79.350] #o

#80-11021
#ligand_binding_atom_basic = [32.632,  -9.769,  77.196] #h
#h_atom_basic = [31.911, -10.430,  77.307] #n

#65-11021
#ligand_binding_atom_basic = [32.970,  -9.473,  77.521] #h
#h_atom_basic = [32.281, -10.176,  77.548] #n

#78-11021
#ligand_binding_atom_basic = [29.407,  -7.967,  78.666] #h
#h_atom_basic = [30.130,  -8.562,  78.618] #o

In [12]:
#59-14010
#ligand_binding_atom_basic = [31.260,  -8.239,  78.881] #h
#h_atom_basic = [30.401,  -8.470,  79.267] #o

#49-14010
#ligand_binding_atom_basic = [30.006,  -7.637,  79.251] #h
#h_atom_basic = [30.155,  -8.570,  79.031] #o

#19-14010
#ligand_binding_atom_basic = [31.730,  -8.433,  76.908] #h
#h_atom_basic = [32.288,  -9.113,  76.391] #n

#20-14010
#ligand_binding_atom_basic = [31.373,  -9.639,  78.550] #h
#h_atom_basic = [30.456,  -9.932,  78.347] #n

#42-14010
#ligand_binding_atom_basic = [31.154,  -8.318,  78.880] #h
#h_atom_basic = [30.391,  -8.863,  79.181] #n

#38_5-14010
#ligand_binding_atom_basic = [31.269,  -7.947,  76.903] #h
#h_atom_basic = [31.236,  -8.754,  76.341] #n

#82_6-14010
#ligand_binding_atom_basic = [31.180,  -7.815,  76.910] #h
#h_atom_basic = [30.573,  -8.572,  76.595] #n

#49_3-14010
#ligand_binding_atom_basic = [30.662,  -8.919,  78.241] #h
#h_atom_basic = [30.165,  -8.491,  78.956] #o

#68_2-14010
#ligand_binding_atom_basic = [30.296,  -7.730,  77.789] #h
#h_atom_basic = [30.258,  -8.696,  77.714] #o

#13_6-14010
#ligand_binding_atom_basic = [31.877,  -8.314,  76.640] #h
#h_atom_basic = [32.505,  -8.825,  76.018] #n

#02_9-14010
ligand_binding_atom_basic = [31.314,  -9.825,  78.525] #h
h_atom_basic = [30.555, -10.427,  78.349] #n

#01_8-14010
#ligand_binding_atom_basic = [30.213,  -8.794,  77.621] #h
#h_atom_basic = [29.928,  -9.665,  77.261] #n

#109_4-14010
#ligand_binding_atom_basic = [31.525,  -6.442,  75.593] #h
#h_atom_basic = [31.521,  -6.892,  74.717] #n

#56_9-14010
#ligand_binding_atom_basic = [31.367,  -8.035,  76.694] #h
#h_atom_basic = [30.732,  -8.766,  76.516] #n

#80_4-14010
#ligand_binding_atom_basic = [29.851,  -7.828,  77.727] #h
#h_atom_basic = [29.761,  -8.598,  77.122] #n

In [13]:
def getTransformationMatrix(angles):
    a, b, g = angles
    m = np.zeros((3, 3), float)
    
    m[0][0] = cos(a)*cos(g) - cos(b)*sin(a)*sin(g)
    m[0][1] = -cos(g)*sin(a) - cos(a)*cos(b)*sin(g)
    m[0][2] = sin(b)*sin(g)
    
    m[1][0] = cos(b)*cos(g)*sin(a) + cos(a)*sin(g)
    m[1][1] = cos(a)*cos(b)*cos(g) - sin(a)*sin(g)
    m[1][2] = -cos(g)*sin(b)
    
    m[2][0] = sin(a)*sin(b)
    m[2][1] = cos(a)*sin(b)
    m[2][2] = cos(b)
    
    return m

In [14]:
def transform_coords(matrix, bias, atom):
    x, y, z = atom
    
    x_transformed = matrix[0][0]*(x-bias[0]) + matrix[0][1]*(y-bias[1]) + matrix[0][2]*(z-bias[2])
    y_transformed = matrix[1][0]*(x-bias[0]) + matrix[1][1]*(y-bias[1]) + matrix[1][2]*(z-bias[2])
    z_transformed = matrix[2][0]*(x-bias[0]) + matrix[2][1]*(y-bias[1]) + matrix[2][2]*(z-bias[2])
    
    transformed_atom = [x_transformed, y_transformed, z_transformed]
    return transformed_atom

In [15]:
def rmsd(x, *args):
    angles = x[:3]
    bias = x[3:]
    lig = args[0]
    
    m = getTransformationMatrix(angles)
      
    deviation = 0
    for k in range(len(ligand)):
        x, y, z = ligand[k]
        deviation += ((m[0][0])*(x-bias[0]) + (m[0][1])*(y-bias[1]) + (m[0][2])*(z-bias[2]) - lig[k][0])**2
        deviation += ((m[1][0])*(x-bias[0]) + (m[1][1])*(y-bias[1]) + (m[1][2])*(z-bias[2]) - lig[k][1])**2
        deviation += ((m[2][0])*(x-bias[0]) + (m[2][1])*(y-bias[1]) + (m[2][2])*(z-bias[2]) - lig[k][2])**2
        
    return rmsdlim - np.sqrt(deviation / len(ligand))

In [16]:
#prot2 = [x for x in protein if x not in bonded_atoms_p]
prot2 = [x for x in protein]

In [17]:
def protein_cons_fun(x, *args):
    angles = x[:3]
    bias = x[3:]
    
    m = getTransformationMatrix(angles)
    
    delta = 2.0
    
    res = []
    for prot_atom in prot2:
        for lig_atom in ligand:
            
            lig_atom_transformed = transform_coords(m, bias, lig_atom)
            distance = dist(prot_atom, lig_atom_transformed)
            res.append(distance - delta)
            
    return res

In [18]:
def hbond_dist(x, *args):
    angles = x[:3]
    bias = x[3:]
    
    m = getTransformationMatrix(angles)
    
    ligand_binding_atom = transform_coords(m, bias, h_atom_basic)
    hbond_dist = dist(ligand_binding_atom, protein_binding_atom)
    
    print("hbond_dist: " + str(hbond_dist))
          
    return hbond_dist

In [19]:
def hbond_dist_cons_fun_ub(x, *args):
    dist_ub = 4.0
    dist = hbond_dist(x)
          
    return dist_ub - dist

In [20]:
def hbond_dist_cons_fun_lb(x, *args):
    dist_lb = 2.0
    dist = hbond_dist(x)
          
    return dist - dist_lb

In [21]:
def hbond_angle(x, *args):
    angles = x[:3]
    bias = x[3:]
    
    m = getTransformationMatrix(angles)
    
    x, y, z = ligand_binding_atom_basic
    x_transformed = (m[0][0])*(x-bias[0]) + (m[0][1])*(y-bias[1]) + (m[0][2])*(z-bias[2])
    y_transformed = (m[1][0])*(x-bias[0]) + (m[1][1])*(y-bias[1]) + (m[1][2])*(z-bias[2])
    z_transformed = m[2][0]*(x-bias[0]) + m[2][1]*(y-bias[1]) + m[2][2]*(z-bias[2])
    
    xh, yh, zh = h_atom_basic
    xh_transformed = (m[0][0])*(xh-bias[0]) + (m[0][1])*(yh-bias[1]) + (m[0][2])*(zh-bias[2])
    yh_transformed = (m[1][0])*(xh-bias[0]) + (m[1][1])*(yh-bias[1]) + (m[1][2])*(zh-bias[2])
    zh_transformed = m[2][0]*(xh-bias[0]) + m[2][1]*(yh-bias[1]) + m[2][2]*(zh-bias[2])
    
    ligand_binding_atom = [x_transformed, y_transformed, z_transformed]
    h_atom = [xh_transformed, yh_transformed, zh_transformed]
    
    hbond_angle = angle_between_three_points(protein_binding_atom, ligand_binding_atom, h_atom)
    
    print("hbond_angle: " + str(arccos(hbond_angle)*180/pi))
          
    return hbond_angle

In [22]:
# initial parameters for optimization
params0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype = float)

# bounds
bnds = Bounds ([-pi, -pi, -pi, -80, -80, -80], [pi, pi, pi, 80, 80, 80])

# constraints
hbond_dist_cons = {'type': 'ineq',
                   'fun': hbond_dist_cons_fun_lb}

hbond_dist_cons2 = {'type': 'ineq',
                   'fun': hbond_dist_cons_fun_ub}

#hbond_angle_cons = {'type': 'ineq',
#                    'fun': hbond_angle_cons_fun}

protein_cons = {'type': 'ineq',
                'fun': protein_cons_fun}

rmsd_cons = {'type': 'ineq',
             'fun': rmsd,
             'args': [ligand]}

rmsdlim = 8

result_dist = sc.optimize.minimize(fun = hbond_dist,
                                       x0 = params0,
                                       method = 'SLSQP', 
                                       jac = '2-point', 
                                       bounds = bnds, 
                                       constraints = (hbond_dist_cons, rmsd_cons, protein_cons),
                                       options={'maxiter': 100, 
                                                'ftol': 0.01, 
                                                'iprint': 1, 
                                                'disp': True, 
                                                'eps': 1.4901161193847656e-08})

angles = result_dist.x[:3]
bias = result_dist.x[3:]

m = getTransformationMatrix(angles)
    
for i in range(len(ligand)):
    atom = ligand[i].copy()
    ligand_copy[i] = transform_coords(m, bias, atom)
    
params0 = result_dist.x

In [23]:
print("hbond_angle: " + str(arccos(hbond_angle(params0))*180/pi))
print("hbond_dist: " + str(hbond_dist(params0)))

hbond_angle: 128.86550275071104
hbond_angle: 128.86550275071104
hbond_dist: 4.409665406808095
hbond_dist: 4.409665406808095


In [24]:
while(hbond_angle(params0) > -0.643 or hbond_dist(params0) > 3.0):
    
    if(hbond_dist(params0) > 3.0):
        rmsdlim = 0.1
        
        rmsd_cons = {'type': 'ineq',
                     'fun': rmsd,
                     'args': [ligand_copy]}
    
        result_dist = sc.optimize.minimize(fun = hbond_dist,
                                           x0 = params0,
                                           method = 'SLSQP', 
                                           jac = '2-point', 
                                           bounds = bnds, 
                                           constraints = (rmsd_cons, protein_cons, hbond_dist_cons2),
                                           options={'maxiter': 20, 
                                                    'ftol': 0.01, 
                                                    'iprint': 1, 
                                                    'disp': True, 
                                                    'eps': 1.4901161193847656e-08})
        
        params0 = result_dist.x
        angles = params0[:3]
        bias = params0[3:]
        m = getTransformationMatrix(angles)
        
        for i in range(len(ligand)):
            atom = ligand[i].copy()
            ligand_copy[i] = transform_coords(m, bias, atom)
        
    if(hbond_angle(params0) > -0.643):
        rmsdlim = 0.1
        
        rmsd_cons = {'type': 'ineq',
                     'fun': rmsd,
                     'args': [ligand_copy]}
    
        result_angle = sc.optimize.minimize(fun = hbond_angle,
                                            x0 = params0,
                                            method = 'SLSQP', 
                                            jac = '2-point', 
                                            bounds = bnds,
                                            constraints = (rmsd_cons, protein_cons, hbond_dist_cons2),
                                            options={'maxiter': 50, 
                                                     'ftol': 0.0001, 
                                                     'iprint': 1, 
                                                     'disp': True, 
                                                     'eps': 1.4901161193847656e-08})
        params0 = result_angle.x
        angles = params0[:3]
        bias = params0[3:]
        m = getTransformationMatrix(angles)
    
        for i in range(len(ligand)):
            atom = ligand[i].copy()
            ligand_copy[i] = transform_coords(m, bias, atom)

hbond_angle: 128.86550275071104
hbond_dist: 4.409665406808095
hbond_dist: 4.409665406808095
hbond_dist: 4.409665406808095
hbond_dist: 4.409665406808095
hbond_dist: 4.409665406808095
hbond_dist: 4.409664931042953
hbond_dist: 4.409666510564554
hbond_dist: 4.409664931042953
hbond_dist: 4.409665410430603
hbond_dist: 4.409665421142681
hbond_dist: 4.4096654049529125
hbond_dist: 4.409665406808095
hbond_dist: 4.409664931042953
hbond_dist: 4.409666510564554
hbond_dist: 4.409664931042953
hbond_dist: 4.409665410430603
hbond_dist: 4.409665421142681
hbond_dist: 4.4096654049529125
hbond_dist: 2.898380227041728
hbond_dist: 2.898380227041728
hbond_dist: 2.898380227041728
hbond_dist: 2.8983797542803917
hbond_dist: 2.898381318197398
hbond_dist: 2.8983797565605123
hbond_dist: 2.898380230281892
hbond_dist: 2.898380241297463
hbond_dist: 2.8983802241573033
hbond_dist: 2.898380227041728
hbond_dist: 2.8983797542803917
hbond_dist: 2.898381318197398
hbond_dist: 2.8983797565605123
hbond_dist: 2.898380230281892
h

hbond_dist: 4.23476857576676
hbond_dist: 4.234768099845629
hbond_dist: 4.234769721306519
hbond_dist: 4.23476809846301
hbond_dist: 4.234768579810393
hbond_dist: 4.23476859004934
hbond_dist: 4.234768574462322
hbond_dist: 6.101347557246936
hbond_dist: 6.101347557246936
hbond_dist: 4.210235331028445
hbond_dist: 4.210235331028445
hbond_dist: 4.225548054166225
hbond_dist: 4.225548054166225
hbond_dist: 4.225548054166225
hbond_dist: 4.225547578861544
hbond_dist: 4.225549190237138
hbond_dist: 4.2255475775133515
hbond_dist: 4.225548058447686
hbond_dist: 4.225548068368219
hbond_dist: 4.225548052745984
hbond_dist: 4.225548054166225
hbond_dist: 4.225547578861544
hbond_dist: 4.225549190237138
hbond_dist: 4.2255475775133515
hbond_dist: 4.225548058447686
hbond_dist: 4.225548068368219
hbond_dist: 4.225548052745984
hbond_dist: 7.463375175795796
hbond_dist: 7.463375175795796
hbond_dist: 4.197050416595619
hbond_dist: 4.197050416595619
hbond_dist: 4.197050416595619
hbond_dist: 4.197049941980523
hbond_dist:

hbond_dist: 4.0079250522512435
hbond_dist: 4.007924576691717
hbond_dist: 4.007926208783735
hbond_dist: 4.007924577250835
hbond_dist: 4.007925056897989
hbond_dist: 4.007925066323107
hbond_dist: 4.007925050690783
hbond_angle: 124.62888668543376
hbond_dist: 4.050934715424697
hbond_angle: 132.05402872953275
hbond_dist: 4.007293928809207
hbond_angle: 132.05402872953275
hbond_angle: 132.05402729035552
hbond_angle: 132.05403336419903
hbond_angle: 132.05402745597624
hbond_angle: 132.0540289673612
hbond_angle: 132.0540287151307
hbond_angle: 132.0540288342395
hbond_dist: 4.007293928809207
hbond_dist: 4.0072934531847695
hbond_dist: 4.007295086123863
hbond_dist: 4.0072934535245075
hbond_dist: 4.007293933313122
hbond_dist: 4.007293942930708
hbond_dist: 4.007293927278659
hbond_angle: 123.80151328823605
hbond_dist: 4.059332244818378
hbond_angle: 131.46659747594305
hbond_dist: 4.006784508844631
hbond_angle: 131.46659747594305
hbond_angle: 131.46659596241832
hbond_angle: 131.46660249578628
hbond_angle:

hbond_dist: 4.008518323253971
hbond_dist: 4.008517850005289
hbond_dist: 4.008519475302953
hbond_dist: 4.008517846897913
hbond_dist: 4.008518327142373
hbond_dist: 4.008518337585222
hbond_dist: 4.008518322012954
hbond_angle: 129.89253124709603
hbond_dist: 4.008807053412709
hbond_angle: 129.88413101492773
hbond_dist: 4.008595440981717
hbond_angle: 129.88413101492773
hbond_angle: 129.88412914454054
hbond_angle: 129.8841375707819
hbond_angle: 129.884129561824
hbond_angle: 129.88413125564037
hbond_angle: 129.8841310082944
hbond_angle: 129.88413110836987
hbond_dist: 4.008595440981717
hbond_dist: 4.008594968003787
hbond_dist: 4.008596592171529
hbond_dist: 4.008594964621413
hbond_dist: 4.008595444871637
hbond_dist: 4.008595455312667
hbond_dist: 4.00859543974196
hbond_angle: 129.89263454371618
hbond_dist: 4.008640608663623
hbond_angle: 129.89263454371618
hbond_angle: 129.89263267365556
hbond_angle: 129.89264118563324
hbond_angle: 129.8926330911315
hbond_angle: 129.89263478443263
hbond_angle: 129

hbond_angle: 129.9271733722739
hbond_dist: 4.008650008009485
hbond_angle: 129.92719087160373
hbond_dist: 4.0086500080102985
hbond_angle: 129.92719262153557
hbond_dist: 4.008650008010398
hbond_angle: 129.92719279652897
hbond_dist: 4.008650008010405
hbond_angle: 129.9271928140281
hbond_dist: 4.00865000801041
hbond_angle: 129.9271928140281
hbond_angle: 129.92719094148154
hbond_angle: 129.92719942429304
hbond_angle: 129.92719136420789
hbond_angle: 129.92719305479426
hbond_angle: 129.927192807183
hbond_angle: 129.92719290740789
hbond_dist: 4.00865000801041
hbond_dist: 4.008649535209907
hbond_dist: 4.008651158764954
hbond_dist: 4.008649531640018
hbond_dist: 4.008650011906467
hbond_dist: 4.0086500223405155
hbond_dist: 4.008650006780195
hbond_angle: 124.57330852890863
hbond_dist: 3.9522483331448224
hbond_angle: 129.16637399579687
hbond_dist: 4.008226077343799
hbond_angle: 129.84915510590508
hbond_dist: 4.0086459361890805
hbond_angle: 129.91936990357175
hbond_dist: 4.008649963127794
hbond_angle

...

hbond_dist: 3.1251179239045253
hbond_dist: 3.1251174482451662
hbond_dist: 3.1251190823017185
hbond_dist: 3.125117446989984
hbond_dist: 3.125117928204051
hbond_dist: 3.1251179381644962
hbond_dist: 3.1251179234442557
hbond_dist: 3.19937385085912
hbond_dist: 3.19937385085912
hbond_dist: 3.1441782760488164
hbond_dist: 3.1441782760488164
hbond_dist: 3.1441782760488164
hbond_dist: 3.144177800548809
hbond_dist: 3.144179434642683
hbond_dist: 3.1441777989703743
hbond_dist: 3.1441782803229317
hbond_dist: 3.144178290318553
hbond_dist: 3.1441782756599794
hbond_dist: 3.1441782760488164
hbond_dist: 3.144177800548809
hbond_dist: 3.144179434642683
hbond_dist: 3.1441777989703743
hbond_dist: 3.1441782803229317
hbond_dist: 3.144178290318553
hbond_dist: 3.1441782756599794
hbond_dist: 3.203793191022695
hbond_dist: 3.203793191022695
hbond_dist: 3.1695856071075657
hbond_dist: 3.1695856071075657
hbond_dist: 3.1695856071075657
hbond_dist: 3.1695851314633425
hbond_dist: 3.1695867659689174
hbond_dist: 3.16958512

hbond_dist: 3.2628820619107755
hbond_dist: 3.262881580123158
hbond_dist: 3.2628831391668793
hbond_dist: 3.262881590928706
hbond_dist: 3.262882068633141
hbond_dist: 3.2628820751491796
hbond_dist: 3.262882060646224
hbond_angle: 124.21133014963797
hbond_dist: 3.1969345696328686
hbond_angle: 138.94072679049142
hbond_dist: 3.2512669854936904
hbond_angle: 138.94072679049142
hbond_angle: 138.94072491572268
hbond_angle: 138.94072728029624
hbond_angle: 138.9407254188522
hbond_angle: 138.94072709197542
hbond_angle: 138.9407267365575
hbond_angle: 138.94072695855684
hbond_dist: 3.2512669854936904
hbond_dist: 3.2512665029783907
hbond_dist: 3.251268076153521
hbond_dist: 3.25126651370427
hbond_dist: 3.251266992002983
hbond_dist: 3.2512669988412
hbond_dist: 3.251266984261726
hbond_angle: 126.03073261428408
hbond_dist: 3.2117005092182045
hbond_angle: 137.85370231826622
hbond_dist: 3.241844569812573
hbond_angle: 137.85370231826622
hbond_angle: 137.85370025814413
hbond_angle: 137.8537034607242
hbond_angl

hbond_dist: 3.2126920842954254
hbond_dist: 3.2126915950876693
hbond_dist: 3.2126932295326385
hbond_dist: 3.212691607977622
hbond_dist: 3.2126920894232414
hbond_dist: 3.2126920982864453
hbond_dist: 3.2126920843332822
hbond_angle: 134.49831461757663
hbond_dist: 3.21317815849104
hbond_angle: 134.49831461757663
hbond_angle: 134.49831130824626
hbond_angle: 134.49831796539314
hbond_angle: 134.49831298965614
hbond_angle: 134.49831494621856
hbond_angle: 134.49831457806636
hbond_angle: 134.4983147286168
hbond_dist: 3.21317815849104
hbond_dist: 3.2131776694046996
hbond_dist: 3.2131793039155174
hbond_dist: 3.21317768217293
hbond_dist: 3.2131781636202144
hbond_dist: 3.2131781724815647
hbond_dist: 3.2131781585289816
hbond_angle: 134.62065868339224
hbond_dist: 3.212040743355137
hbond_angle: 134.62065868339224
hbond_angle: 134.62065538963756
hbond_angle: 134.62066222265105
hbond_angle: 134.6206570733377
hbond_angle: 134.62065901234715
hbond_angle: 134.6206586430696
hbond_angle: 134.62065879426177
hbo

hbond_dist: 3.1014108584287454
hbond_dist: 3.101410391024544
hbond_dist: 3.1014120168619876
hbond_dist: 3.1014103831905886
hbond_dist: 3.1014108637980295
hbond_dist: 3.1014108723289104
hbond_dist: 3.101410858457659
hbond_dist: 3.0403472553917386
hbond_dist: 3.0403472553917386
hbond_dist: 3.0696498168529844
hbond_dist: 3.0696498168529844
hbond_dist: 3.0696498168529844
hbond_dist: 3.0696493531780957
hbond_dist: 3.069650969144297
hbond_dist: 3.06964934017072
hbond_dist: 3.069649822006566
hbond_dist: 3.0696498308330886
hbond_dist: 3.0696498170574853
hbond_dist: 3.0696498168529844
hbond_dist: 3.0696493531780957
hbond_dist: 3.069650969144297
hbond_dist: 3.06964934017072
hbond_dist: 3.069649822006566
hbond_dist: 3.0696498308330886
hbond_dist: 3.0696498170574853
hbond_dist: 3.0986102295396205
hbond_dist: 3.0986102295396205
hbond_dist: 3.0851982573904513
hbond_dist: 3.0851982573904513
hbond_dist: 3.0851982573904513
hbond_dist: 3.0851977932118753
hbond_dist: 3.085199410906953
hbond_dist: 3.08519

hbond_dist: 2.7583488119225246
hbond_dist: 2.7583483537793656
hbond_dist: 2.758349951953304
hbond_dist: 2.7583483356438396
hbond_dist: 2.758348816836861
hbond_dist: 2.758348825984834
hbond_dist: 2.758348811541263
hbond_dist: 2.9087636880740604
hbond_dist: 2.9087636880740604
hbond_dist: 2.801619021446618
hbond_dist: 2.801619021446618
hbond_dist: 2.801619021446618
hbond_dist: 2.8016185616911096
hbond_dist: 2.8016201667615066
hbond_dist: 2.801618545884199
hbond_dist: 2.801619026631019
hbond_dist: 2.8016190354116373
hbond_dist: 2.8016190210661165
hbond_dist: 2.801619021446618
hbond_dist: 2.8016185616911096
hbond_dist: 2.8016201667615066
hbond_dist: 2.801618545884199
hbond_dist: 2.801619026631019
hbond_dist: 2.8016190354116373
hbond_dist: 2.8016190210661165
hbond_dist: 2.917641979760939
hbond_dist: 2.917641979760939
hbond_dist: 2.917641979760939
hbond_dist: 2.9176415179462345
hbond_dist: 2.9176431304813186
hbond_dist: 2.917641504779502
hbond_dist: 2.9176419853906963
hbond_dist: 2.9176419935

In [25]:
print(params0)

[ 0.55780644 -0.08034091 -0.48884112  3.8072548   5.96116272  0.03364812]


In [26]:
angles = params0[:3]
bias = params0[3:]

m = getTransformationMatrix(angles)

for i in range(len(ligand)):
        atom = ligand[i].copy()
        ligand_copy2[i] = transform_coords(m, bias, atom)

In [27]:
generateNewPDB(path_to_ligand, ligand_copy2)